## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-09-07 03:51:26,light rain,-23.1203,-134.9692,72.86,76,100,25.97
1,1,Saint George,US,2022-09-07 03:56:23,broken clouds,37.1041,-113.5841,99.63,16,53,4.76
2,2,Shamkhal,RU,2022-09-07 03:57:03,light rain,43.0596,47.3373,59.02,90,100,6.31
3,3,Hobart,AU,2022-09-07 03:54:01,broken clouds,-42.8794,147.3294,57.06,64,75,10.36
4,4,Busselton,AU,2022-09-07 03:57:04,broken clouds,-33.6500,115.3333,62.53,59,83,10.25


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 85
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Current Description,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Saint George,US,2022-09-07 03:56:23,broken clouds,37.1041,-113.5841,99.63,16,53,4.76
38,38,Bikaner,IN,2022-09-07 03:57:16,few clouds,28.0167,73.3000,91.76,40,12,4.52
48,48,Lantawan,PH,2022-09-07 03:58:19,overcast clouds,10.6167,123.9833,85.69,72,100,5.10
52,52,Mount Isa,AU,2022-09-07 03:54:11,clear sky,-20.7333,139.5000,91.17,15,4,14.97
66,66,Mecca,SA,2022-09-07 03:58:25,scattered clouds,21.4267,39.8261,87.06,48,45,4.81
76,76,Adrar,MR,2022-09-07 03:58:28,broken clouds,20.5022,-10.0711,93.18,17,62,13.35
77,77,Jackson,US,2022-09-07 03:58:28,broken clouds,42.4165,-122.8345,88.75,38,57,1.48
85,85,Nishihara,JP,2022-09-07 03:58:30,scattered clouds,26.1842,127.7558,85.28,62,40,17.27
88,88,Shimoda,JP,2022-09-07 03:58:31,overcast clouds,34.6667,138.9500,86.36,84,99,2.77
112,112,Huilong,CN,2022-09-07 03:59:39,clear sky,31.8111,121.6550,85.21,57,2,6.69


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 1 to 700
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              77 non-null     int64  
 1   City                 77 non-null     object 
 2   Country              77 non-null     object 
 3   Date                 77 non-null     object 
 4   Current Description  77 non-null     object 
 5   Lat                  77 non-null     float64
 6   Lng                  77 non-null     float64
 7   Max Temp             77 non-null     float64
 8   Humidity             77 non-null     int64  
 9   Cloudiness           77 non-null     int64  
 10  Wind Speed           77 non-null     float64
dtypes: float64(4), int64(3), object(4)
memory usage: 7.2+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(how = 'any', inplace = True)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint George,US,99.63,broken clouds,37.1041,-113.5841,
38,Bikaner,IN,91.76,few clouds,28.0167,73.3000,
48,Lantawan,PH,85.69,overcast clouds,10.6167,123.9833,
52,Mount Isa,AU,91.17,clear sky,-20.7333,139.5000,
66,Mecca,SA,87.06,scattered clouds,21.4267,39.8261,
76,Adrar,MR,93.18,broken clouds,20.5022,-10.0711,
77,Jackson,US,88.75,broken clouds,42.4165,-122.8345,
85,Nishihara,JP,85.28,scattered clouds,26.1842,127.7558,
88,Shimoda,JP,86.36,overcast clouds,34.6667,138.9500,
112,Huilong,CN,85.21,clear sky,31.8111,121.6550,


In [8]:
# Parameters
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = 'any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint George,US,99.63,broken clouds,37.1041,-113.5841,Best Western Plus Abbey Inn
38,Bikaner,IN,91.76,few clouds,28.0167,73.3000,The Laxmi Niwas Palace
48,Lantawan,PH,85.69,overcast clouds,10.6167,123.9833,Durano Ecofarm and Spring Resort
52,Mount Isa,AU,91.17,clear sky,-20.7333,139.5000,ibis Styles Mt Isa Verona
66,Mecca,SA,87.06,scattered clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
...,...,...,...,...,...,...,...
667,Taitung,TW,89.87,broken clouds,22.7583,121.1444,Taitung Bali Suites Hotel
677,Dicabisagan,PH,88.61,clear sky,17.0818,122.4157,Nancy Montanes Bernardo
695,Sadao,TH,85.82,overcast clouds,6.6388,100.4234,Banana Resort Sadao
699,Kudat,MY,87.73,few clouds,6.8837,116.8477,Dream Garden Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'weather_data/weatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
    <dl>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Weather Description</dt><dd>{Current Description}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))